In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import cv2
import numpy as np
import argparse
from tqdm.notebook import tqdm
from PIL import Image
from pathlib import Path
import os
import time
import clip

In [2]:
# --- 1. SET YOUR LOCAL DIRECTORIES ---
# We are in the .../visual_model/ folder, so we go up one level
os.chdir("..")
print(f"Working directory set to: {os.getcwd()}")

# Path(".") is now your main project folder (e.g., 'context-aware-video-retrieval')
INPUT_DIR = Path(".") / "media"
OUTPUT_DIR = Path(".") / "embeddings_out" / "video2048"

# --- 2. SET YOUR MODEL PARAMETERS ---
FRAME_SAMPLE_RATE = 30
BATCH_SIZE = 32 # (You can try 64 for CLIP if you have enough VRAM)
CLIP_MODEL_NAME = "ViT-B/32"

# --- 3. DEFINE VIDEO EXTENSIONS TO FIND ---
VIDEO_EXTENSIONS = [".mp4", ".mov", ".avi", ".mkv", ".webm"]

Working directory set to: c:\Users\Shanette\Downloads\COLLEGE\CSST Y4-T1\THS-ST2\context-aware-video-retrieval


In [3]:
print("Setting up model and device...")

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load(CLIP_MODEL_NAME, device=device)
model.eval()
print(f"Using device: {device}")

Setting up model and device...
Using device: cpu


In [4]:
def extract_clip_embeddings(
    video_path: Path, 
    model, 
    preprocess, 
    device: str, 
    frame_sample_rate: int = 30, 
    batch_size: int = 32
) -> np.ndarray:
    """Extracts mean-pooled CLIP visual embeddings from a single video file."""
    if not video_path.exists():
        raise FileNotFoundError(f"Video file not found: {video_path}")

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise IOError(f"Cannot open video file: {video_path}")

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    all_features = []
    frame_batch = []
    frame_idx = 0

    pbar = tqdm(total=frame_count, desc=f"Frames for {video_path.name}", leave=False)

    with torch.no_grad():
        while True:
            ret, frame = cap.read()
            if not ret: break
            pbar.update(1)

            if frame_idx % frame_sample_rate == 0:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                pil_img = Image.fromarray(frame_rgb)
                frame_batch.append(pil_img)

                if len(frame_batch) == batch_size:
                    image_inputs = torch.stack(
                        [preprocess(img) for img in frame_batch]
                    ).to(device)
                    image_features = model.encode_image(image_inputs)
                    image_features /= image_features.norm(dim=-1, keepdim=True)
                    all_features.append(image_features.cpu())
                    frame_batch = []
            frame_idx += 1

        if frame_batch:
            image_inputs = torch.stack(
                [preprocess(img) for img in frame_batch]
            ).to(device)
            image_features = model.encode_image(image_inputs)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            all_features.append(image_features.cpu())

    cap.release()
    pbar.close()
    if not all_features:
        raise ValueError(f"No frames sampled for {video_path.name}")

    embeddings = torch.cat(all_features)
    mean_embedding = embeddings.mean(dim=0).numpy()
    return mean_embedding

In [5]:
# Create the output directory if it doesn't exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Reading videos from: {INPUT_DIR.resolve()}")
print(f"Saving embeddings to: {OUTPUT_DIR.resolve()}")

# Find all video files
video_files = []
for ext in VIDEO_EXTENSIONS:
    video_files.extend(INPUT_DIR.glob(f"*{ext}"))
print(f"Found {len(video_files)} videos.")

# Get list of files ALREADY in the output folder to skip them
existing_embeddings = {f.name for f in OUTPUT_DIR.glob('*_clip.npy')}
print(f"Found {len(existing_embeddings)} existing CLIP embeddings.")

for video_path in tqdm(video_files, desc="Processing Videos (CLIP)"):
    output_filename = f"{video_path.stem}_clip.npy"

    # Skip if already processed
    if output_filename in existing_embeddings:
        continue
    
    output_path = OUTPUT_DIR / output_filename
    
    try:
        print(f"Processing {video_path.name}...")
        mean_embedding = extract_clip_embeddings(
            video_path=video_path,
            model=model,
            preprocess=preprocess,
            device=device,
            frame_sample_rate=FRAME_SAMPLE_RATE,
            batch_size=BATCH_SIZE
        )
        np.save(output_path, mean_embedding)

    except Exception as e:
        print(f"\n[ERROR] Failed to process {video_path.name}: {e}")

print("\n--- Batch processing complete. ---")

Reading videos from: C:\Users\Shanette\Downloads\COLLEGE\CSST Y4-T1\THS-ST2\context-aware-video-retrieval\media
Saving embeddings to: C:\Users\Shanette\Downloads\COLLEGE\CSST Y4-T1\THS-ST2\context-aware-video-retrieval\embeddings_out\video2048
Found 100 videos.
Found 0 existing CLIP embeddings.


Processing Videos (CLIP):   0%|          | 0/100 [00:00<?, ?it/s]

Processing airball_1.mp4...


Frames for airball_1.mp4:   0%|          | 0/300 [00:00<?, ?it/s]

Processing airball_10.mp4...


...

Processing where_7.mp4...


Frames for where_7.mp4:   0%|          | 0/641 [00:00<?, ?it/s]

Processing where_8.mp4...


Frames for where_8.mp4:   0%|          | 0/487 [00:00<?, ?it/s]


--- Batch processing complete. ---
